# Read data example

In [2]:
import os
import typing
from tqdm import tqdm
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json

from keras.utils import  Sequence



In [3]:
ROOT_DATA_FOLDER = r"/kaggle/input/drive-redused"

TRAIN_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, r"YaCupTrain")

TEST_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, r"YaCupTest")
label_columns = ['x', 'y', 'yaw']

## Read Data

In [4]:

# Load all ids of a dataset

def read_testcase_ids(dataset_path: str):
    ids = [int(case_id) for case_id in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, case_id))]
    return ids

ids = read_testcase_ids(TRAIN_DATASET_PATH)
len(ids)

train_ids = np.random.choice(ids, size=round(0.70*len(ids)), replace=False, p=None)
test_ids = [el for el in ids if el not in train_ids]


class DataFilePaths:
    def __init__(self, testcase_path: str):
        self.testcase_path = testcase_path

    def localization(self):
        return os.path.join(self.testcase_path, 'localization.csv')

    def control(self):
        return os.path.join(self.testcase_path, 'control.csv')

    def metadata(self):
        return os.path.join(self.testcase_path, 'metadata.json')

    # exists only for test_dataset
    def requested_stamps(self):
        return os.path.join(self.testcase_path, 'requested_stamps.csv')


def read_localization(localization_path: str):
    return pd.read_csv(localization_path)

def read_control(control_path):
    return pd.read_csv(control_path)

def read_metadata(metadata_path: str):
    with open(metadata_path, 'r') as f:
        data = json.load(f)
    return data

def read_requested_stamps(requested_stamps_path: str):
    return pd.read_csv(requested_stamps_path)

def read_testcase(dataset_path: str, testcase_id: str, is_test: bool = False):
    testcase_path = os.path.join(dataset_path, str(testcase_id))
    data_file_paths = DataFilePaths(testcase_path)

    testcase_data = {}
    testcase_data['localization'] = read_localization(data_file_paths.localization())
    testcase_data['control'] = read_control(data_file_paths.control())
    testcase_data['metadata'] = read_metadata(data_file_paths.metadata())
    if is_test:
        testcase_data['requested_stamps'] = read_requested_stamps(data_file_paths.requested_stamps())

    return testcase_data


def read_testcases(dataset_path: str, is_test: bool = False, testcase_ids: typing.Iterable[int] = None):
    result = {}
    if testcase_ids is None:
        testcase_ids = read_testcase_ids(dataset_path)

    for testcase_id in tqdm(testcase_ids):
        testcase = read_testcase(dataset_path, testcase_id, is_test=is_test)
        result[testcase_id] = testcase
    return result

In [5]:
train_dataset = read_testcases(TRAIN_DATASET_PATH)
len(train_dataset)

100%|██████████| 3884/3884 [01:38<00:00, 39.35it/s]


3884

In [6]:

train = {k: v for k, v in train_dataset.items() if k in train_ids}
test = {k: v for k, v in train_dataset.items() if k in test_ids}


# Window Generate

## 1 dict to df + mixed bach size

In [7]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train, val,
               label_columns=None):
    # Store the raw data.
    self.train = train
    self.val = test

    train_df = self.dict_to_df(train)
    val_df = self.dict_to_df(test)



    
    

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]


  def dict_to_df(self, dict_df):
      for i in dict_df.keys():
          return dict_df[i]['localization']
      

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

## 2 dict to df + mixed bach size logic

In [ ]:
class WindowGenerator(Sequence):
  def __init__(self, input_width, label_width, shift,
               train, val,
               label_columns=None, batch_size=32):
    # Store the raw data.
    self.train = train
    self.val = test

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
    self.batch_size =  batch_size
    buch_for_buch = round(buch_data_len - label_width/shift) # quality of el from dict with we use for one buch



  def __len__(self):
    'Denotes the number of batches per epoch'
    len_after_windows = train.keys()*buch_data_len/input_width*shift
    return int(np.floor(len(len_after_windows) / self.batch_size))

    
  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.)) #!!!!!
    if self.shuffle == True:
        np.random.shuffle(self.indexes)


  def __getitem__(self, index):
        'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [k for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y


   def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_data = list() 
        batch_labels = list()
        
        # Generate data
        for i in train.keys():
            # Store sample

            batch_data.append()
            #Store class
            batch_labels.append(label)
            
        return np.array(batch_data), np.array(batch_labels)
      
  def dict_to_df(self, dict_df):
      for i in dict_df.keys():
          return dict_df[i]['localization']


  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
           axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  def bach_prepare():
      
      

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

## With UP Class Logic

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df


    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])




class UPWindowGenerator():
    def __init__(train_dict, val_dict,):
        self.train_dict = train_dict
        self.batch_size =  batch_size
        #self.val_df = val_df

    def get_items():
        bach 
        for i in train_dict.keys():
            
            WindowGenerator(train_dict[i]['localization'])



## step by step logic

In [8]:
class WindowGenerator():
  def __init__(self,train_df, input_width, label_width, shift,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df


    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])






In [9]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels


WindowGenerator.split_window = split_window

In [60]:
train_seq_X = []
train_seq_y = []

for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)

    train_seq_X.append(X)
    train_seq_y.append(y)



In [62]:
A = np.array(train_seq_X)
A.shape

(2719, 3, 5, 7)

In [10]:
train_seq_X = []
train_seq_y = []

for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)

    for X_i, y_i in zip(X , y):

         train_seq_X.append(X_i)
         train_seq_y.append(y_i)
    



In [23]:
train_seq_X= np.array(train_seq_X)
train_seq_y = np.array(train_seq_y)
## Add normalize

In [24]:
train_seq_X.shape

(8157, 5, 7)

In [25]:
train_seq_y.shape

(8157, 5, 3)

In [11]:
B = np.array(train_seq_X)
B.shape

(8157, 5, 7)

In [ ]:
inputs = np.random.random((32, 10, 8))
>>> lstm = keras.layers.LSTM(4)
>>> output = lstm(inputs)
>>> output.shape
(32, 4)
>>> lstm = keras.layers.LSTM(
...     4, return_sequences=True, return_state=True)
>>> whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)
>>> whole_seq_output.shape
(32, 10, 4)
>>> final_memory_state.shape
(32, 4)
>>> final_carry_state.shape
(32, 4)

In [50]:
inputs = np.random.random((32, 5, 7))
lstm = keras.layers.LSTM(3, return_sequences=True, return_state=True)
whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)
whole_seq_output.shape,




(TensorShape([32, 5, 3]),)

In [66]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(3, return_sequences=True)
    # Shape => [batch, time, features]
    #tf.keras.layers.Dense(units=5,3)
])


In [17]:
import keras

In [86]:
lstm_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SquaredHinge(),
    metrics=[
        keras.metrics.Accuracy(), keras.metrics.RootMeanSquaredError(), keras.metrics.CosineSimilarity()
    ],
)


In [ ]:
losses.Tversky()
SquaredHinge

In [87]:
lstm_model.fit(train_seq_X, train_seq_y, epochs=20, batch_size=2028)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 580ms/step - accuracy: 0.0000e+00 - cosine_similarity: -0.4171 - loss: 5262706.0000 - root_mean_squared_error: 11641.0127
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 595ms/step - accuracy: 0.0000e+00 - cosine_similarity: -0.4143 - loss: 5170926.0000 - root_mean_squared_error: 11829.5566
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 583ms/step - accuracy: 0.0000e+00 - cosine_similarity: -0.4117 - loss: 5117136.5000 - root_mean_squared_error: 11869.7197
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 580ms/step - accuracy: 0.0000e+00 - cosine_similarity: -0.4149 - loss: 5143760.0000 - root_mean_squared_error: 11630.1875
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 686ms/step - accuracy: 0.0000e+00 - cosine_similarity: -0.4135 - loss: 5178601.5000 - root_mean_squared_error: 11725.2979
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 578ms/step - accuracy: 0.0000e+00 - cosine_similarity: -0.4134 - loss: 5189556.5000 - root_mean_squared_error: 11690.2676
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━

In [14]:
lstm_model.compile(train_seq_X, train_seq_y)

<Sequential name=sequential, built=False>

In [47]:
train_seq_X = tf.zeros([5,7,1],dtype=tf.dtypes.float64,)
train_seq_y = tf.zeros([5,3,1],dtype=tf.dtypes.float64,)
print(train_seq_X.ndim, train_seq_X.shape)
for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)
    #print(X)
    for X_i, y_i in zip(X , y):
        #print(X_i.ndim, X_i.shape)
        train_seq_X = tf.concat([train_seq_X,X_i], 1 )
        train_seq_y =  tf.concat([train_seq_y,y_i], 1 )



3 (5, 7, 1)


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [5,7,1] vs. shape[1] = [5,7] [Op:ConcatV2] name: concat

In [22]:
train_seq_X.shape()

AttributeError: 'list' object has no attribute 'shape'

In [ ]:

print(train_seq_X.ndim, train_seq_X.shape)
for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)
    #print(X)
    train_seq_X = tf.zeros([5,7],dtype=tf.dtypes.float64,)
    train_seq_y = tf.zeros([5,3],dtype=tf.dtypes.float64,)
    for X_i, y_i in zip(X , y):
        #print(X_i.ndim, X_i.shape)
        train_seq_X = tf.concat([train_seq_X,X_i], 0 )
        train_seq_y =  tf.concat([train_seq_y,y_i], 0 )

train_seq_X

In [34]:
train_seq_X.shape

(285495,)

In [12]:
train_seq_y

<tf.Tensor: shape=(40790, 3), dtype=float64, numpy=
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 2.20955557e+04, -6.52065807e+04,  2.06363593e+00],
       [ 2.20954131e+04, -6.52063152e+04,  2.06305254e+00],
       [ 2.20952699e+04, -6.52060482e+04,  2.06283024e+00]])>

In [21]:
train_seq_X = np.array([])

In [22]:
train_seq_X

array([], dtype=float64)

In [26]:

train_seq_X = np.array([[]])
#print(train_seq_X.ndim, train_seq_X.shape)
#train_seq_y = np.array([])
for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)
    #print(X)
    for X_i, y_i in zip(X , y):
        #print(X_i.ndim, X_i.shape)
        train_seq_X = np.append(train_seq_X,X_i) #,axis=0)
        #train_seq_y = np.append(train_seq_y,y_i, axis=0)

train_seq_X

array([ 0.00000000e+00, -3.51938598e+03, -1.39464479e+04, ...,
        5.09623448e-03, -4.82397264e-03,  2.06305254e+00])

In [68]:
train_seq_X

array([[[6.76819439e-310, 6.76819439e-310, 4.93452746e-310,
         4.93452746e-310, 0.00000000e+000, 0.00000000e+000,
         4.93451962e-310],
        [4.93451962e-310, 5.43472210e-323, 0.00000000e+000,
         3.95252517e-323, 8.48798316e-314, 3.95252517e-323,
         0.00000000e+000],
        [2.47032823e-323, 2.12199584e-314, 0.00000000e+000,
         2.47032823e-323, 1.23516411e-322, 0.00000000e+000,
         2.47032823e-323],
        [4.24399163e-314, 0.00000000e+000, 2.47032823e-323,
         4.24399163e-314, 0.00000000e+000, 2.47032823e-323,
         6.36598742e-314],
        [0.00000000e+000, 2.47032823e-323, 5.08887615e-322,
         0.00000000e+000, 2.47032823e-323, 8.48798321e-314,
         0.00000000e+000],
        [2.47032823e-323, 2.12199587e-314, 0.00000000e+000,
         2.47032823e-323, 1.06099790e-313, 0.00000000e+000,
         2.47032823e-323],
        [1.06099790e-313, 0.00000000e+000, 2.47032823e-323,
         2.33419537e-313, 0.00000000e+000, 2.47032823e-323

In [23]:
len(train_seq_y)

122355

In [21]:
train_seq_X

array([ 0.00000000e+00, -3.51938598e+03, -1.39464479e+04, ...,
        3.81686862e-02, -2.76660030e-02, -7.57760663e-01])

In [ ]:
w2 = WindowGenerator(train=train, val=test, label_columns=['x','y','yaw'], input_width=5, label_width=5, shift=1)

In [ ]:
w2

In [ ]:
w2 = WindowGenerator(train_df=train[3]['localization'], val_df=test[2]['localization'], label_columns=['x','y','yaw'], input_width=5, label_width=5, shift=1)

In [ ]:
example_window = tf.stack([np.array(train[3]['localization'][:w2.total_window_size]),
                           np.array(train[3]['localization'][100:100+w2.total_window_size]),
                           np.array(train[3]['localization'][200:200+w2.total_window_size])
                           ])



In [ ]:
example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')


In [ ]:
def make_dataset(self, data):
    data = np.array(el, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
len(train)

In [ ]:
@property
def train(self):
    for i in train_ids:
        train_df = train[i]['localization']
        return self.make_dataset(self.train_df)



@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val

In [ ]:
!install paramiko

In [ ]:
wide_window = WindowGenerator( train_df=train[3]['localization'], val_df=test[2]['localization'], label_columns=['x','y','yaw'],
    input_width=24, label_width=24, shift=1,)

wide_window

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
history = compile_and_fit(lstm_model, w2)

In [ ]:
train_dataset[5]['control']['stamp_ns'] = train_dataset[5]['control']['stamp_ns']/10e9
train_dataset[5]['localization']['stamp_ns'] = train_dataset[5]['localization']['stamp_ns']/10e9

In [ ]:
train_df = pd.DataFrame()
for el in train_dataset:
    

In [ ]:
train_dataset[25]['control']

In [ ]:
train_dataset[25]['control'].drop(columns='stamp_ns')

In [ ]:
pd.concat([df1.set_index('A'),df2.set_index('A')], axis=1, join='inner').reset_index()


In [ ]:
a = train_dataset[25]['localization'][74:1574]

In [ ]:
train_dataset[25]['control'].drop(columns='stamp_ns')

In [ ]:
pd.concat([train_dataset[25]['localization'][74:1574].reset_index(),train_dataset[25]['control'].drop(columns='stamp_ns').reset_index()], axis=1).reset_index().drop(columns=['index','level_0'])

In [ ]:
test_dataset = read_testcases(TEST_DATASET_PATH, is_test=True)
len(test_dataset)

In [ ]:
loc stams x y z roll pitch yaw 0--> LSTM --> next 1 loc stams x y z roll pitch yaw

In [ ]:
[el.values() for el in test_dataset.values()]

In [ ]:
# LSTM conv 1d

In [ ]:
from keras.layers.recurrent import LSTM

In [ ]:
from tensorflow.python.keras.layers.recurrent import LSTM, Bidirectional

In [ ]:
model = Sequential([
    LSTM(50, activation='tanh', return_sequences=True, input_shape=(10, 5)),  # First LSTM layer
    LSTM(30, activation='tanh'),  # Second LSTM layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
tf.keras.layers.Bidirectional(
    layer, merge_mode='concat'
)


In [ ]:
class Dataprepare():
    def __init__(dataset):
        self.dataset = dataset


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        current_target = self.target[idx]
        current_tweet = self.tweets[idx]
        sequence = []
        for word in current_tweet:
            if word in self.word_to_idx.keys():
                sequence.append(self.word_to_idx[word])

        return {
                'x': torch.tensor(sequence, dtype=torch.long),
                'y': torch.tensor(current_target, dtype=torch.long)
            }


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Dummy baseline

### read test dataset

In [ ]:
import numpy as np

NSECS_IN_SEC = 1000000000

def secs_to_nsecs(secs: float):
    return int(secs * NSECS_IN_SEC)

def nsecs_to_secs(nsecs: int):
    return float(nsecs) / NSECS_IN_SEC

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

In [ ]:
pip install keras==2.10.0

In [ ]:
import numpy as np
np.__version__

### simple pose prediction logic without taking into account control states

In [ ]:
def localization_df_to_poses(loc, contrl):
    loc_contrl = pd.concat([loc[74:1574].reset_index(),contrl.drop(columns='stamp_ns').reset_index()], axis=1).reset_index().drop(columns=['index','level_0'])
    poses = []
    for stamp_ns, x, y, yaw,acceleration_level, steering, in zip(loc_contrl['stamp_ns'], loc_contrl['x'], loc_contrl['y'], loc_contrl['yaw'], loc_contrl['acceleration_level'],loc_contrl['steering']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses


In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

In [ ]:
localization_df_to_poses(loc_df)


In [ ]:

import numpy as np

In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

# naive estimation of speed at last known localization pose
def dummy_estimate_last_speed(localization_poses):
    last_pose = localization_poses[-1]

    start_pose_idx = -1
    for i, pose in enumerate(localization_poses, start=1-len(localization_poses)):
        start_pose_idx = i
        if nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(pose['stamp_ns']) > 1.: # sec
            break

    start_pose = localization_poses[start_pose_idx]
    dt_sec = nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(start_pose['stamp_ns'])

    if dt_sec > 1e-5:
        return np.linalg.norm(last_pose['pos'][:2] - start_pose['pos'][:2]) / dt_sec
    return 5. # some default value

def dummpy_predict_pose(last_loc_pose: dict, last_speed: float, prediction_stamp: int):
    dt_sec = nsecs_to_secs(prediction_stamp) - nsecs_to_secs(last_loc_pose['stamp_ns'])
    distance = dt_sec * last_speed
    direction = yaw_direction(last_loc_pose['yaw'])
    pos_translate = direction * distance
    return {"pos": last_loc_pose['pos'] + pos_translate, 'yaw': last_loc_pose['yaw']}

In [ ]:
def predict_testcase(testcase: dict):
    loc_df = testcase['localization']
    localization_poses = localization_df_to_poses(loc_df)

    last_loc_pose = localization_poses[-1]
    last_speed = dummy_estimate_last_speed(localization_poses)

    predicted_poses = []
    for stamp in testcase['requested_stamps']['stamp_ns']:
        pose = dummpy_predict_pose(last_loc_pose, last_speed, stamp)
        predicted_poses.append(pose)

    predictions = {}
    predictions['stamp_ns'] = testcase['requested_stamps']['stamp_ns']
    predictions['x'] = [pose['pos'][0] for pose in predicted_poses]
    predictions['y'] = [pose['pos'][1] for pose in predicted_poses]
    predictions['yaw'] = [pose['yaw'] for pose in predicted_poses]
    return pd.DataFrame(predictions)

def predict_test_dataset(test_dataset: dict):
    predictions = {}
    for testcase_id, testcase in tqdm(test_dataset.items()):
        predictions[testcase_id] = predict_testcase(testcase)
    return predictions

### make prediction for requested stamps

In [ ]:
predictions = predict_test_dataset(train_dataset)
len(predictions)

In [ ]:
test_predictions = predict_test_dataset(test_dataset)
len(test_predictions)

In [ ]:
test_predictions

### write predictions

In [ ]:
def write_predictions(dataset_predictions: dict, prediction_file_path: str):
    prediction_list = []
    for testcase_id, prediction in tqdm(dataset_predictions.items()):
        prediction['testcase_id'] = [testcase_id] * len(prediction)
        prediction_list.append(prediction)
    predictions_df = pd.concat(prediction_list)
    predictions_df = predictions_df.reindex(columns=["testcase_id", "stamp_ns", "x", "y", "yaw"])
    print(len(predictions_df))
    predictions_df.to_csv(prediction_file_path, index=False, header=True)

In [ ]:
write_predictions(test_predictions, os.path.join(ROOT_DATA_FOLDER, "predictions.csv"))

# Calculate metric

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.


### implementation of the metric calculation

In [ ]:
import numpy as np
import pandas as pd

SEGMENT_LENGTH = 1.

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

def build_car_points(x_y_yaw):
    directions = np.vstack(yaw_direction(x_y_yaw[:, -1]))

    front_points = x_y_yaw[:, :-1] + SEGMENT_LENGTH * directions.T
    points = np.vstack([x_y_yaw[:, :-1], front_points])
    return points

def build_car_points_from_merged_df(df: pd.DataFrame):
    points_gt = df[['x_gt', 'y_gt', 'yaw_gt']].to_numpy()
    points_pred = df[['x_pred', 'y_pred', 'yaw_pred']].to_numpy()

    points_gt = build_car_points(points_gt)
    points_pred = build_car_points(points_pred)
    return points_gt, points_pred

def calculate_metric_testcase(df: pd.DataFrame):
    points_gt, points_pred = build_car_points_from_merged_df(df)

    metric = np.mean(np.sqrt(2. * np.mean((points_gt - points_pred) ** 2, axis=1)))
    return metric

def calculate_metric_dataset(ground_truth_df: pd.DataFrame, prediction_df: pd.DataFrame):
    assert (len(ground_truth_df) == len(prediction_df))

    df = ground_truth_df.merge(prediction_df, on=['testcase_id', 'stamp_ns'], suffixes=['_gt', '_pred'])

    metric = df.groupby('testcase_id').apply(calculate_metric_testcase)
    return np.mean(metric)